# Preprocesamiento de hilos en 4chan

En este cuaderno se procesan las publicaciones de 4chan, obtenidas a través de [web scrapping](./Scraper-4chan). Estas publicaciones pertenecen a múltiples [hilos que son considerados pertinentes]() para el objetivo de la investigación, ya sea por su idiosincrática nocividad en el uso del lenguaje, por la expresión misma de ideaciones/intenciones suicidas, o por contener sugerencias para llevar a cabo el suicidio.

El objetivo de este cuaderno es retirar caracteres inútiles, transformar el contenido de cada publicación a minúsculas y eliminar los saltos de lí­nea en cada publicación. Además, es posible emplear una lista de palabras objetivo y localizar las publicaciones que contengan alguna de ellas.

Para esta tarea se utilizará la librería de `pandas`, pues su parser de archivos `.csv` admite por defecto el uso de saltos de lí­nea en los campos de texto.

In [1]:
import fnmatch
import pandas
import os
import re

A raíz de que se han consultado (y se consultarán) múltiples hilos, este script lee todas las publicaciones de los hilos y realiza múltiples operaciones sobre ellos.

A su vez, se crea una lista de nombres de archivos ya procesados, para excluirlos de la lectura cuando se trabaje en sesiones posteriores.

In [2]:
# Función para encontrar archivos

def encontrar_archivos(patron: str, ruta_base: str, lista_ex: list, verbose: bool = False):

    archivos = []
    index = 1

    for root, _, files in os.walk(ruta_base):

        for name in files:

            if fnmatch.fnmatch(name, patron) and name not in lista_ex:
                archivos.append(os.path.join(root, name))
                if verbose:
                    print(f'Archivo #{index}: {name}')
                    index += 1

    return archivos

In [3]:
# Leemos los archivos ya procesados desde un archivo de texto
archivos_procesados = []

with open('./datos/lista_procesados.txt', 'r') as archivo: 
    for linea in archivo:
        archivos_procesados.append(linea.strip())

archivos_de_la_sesion = []
archivos_procesados[0:5]

['ahorcarme_complete.csv',
 'colgarme_complete.csv',
 '',
 '1436774 - comments & replies.csv',
 '17913796 - comments & replies.csv']

In [4]:
rutas_archivos = []
rutas_base = [
    './datos/hilos_4chan',
]

for ruta in rutas_base:
    rutas_archivos.extend(
        encontrar_archivos(
            patron='*.csv',
            ruta_base=ruta,
            lista_ex=archivos_procesados,
            verbose=True
        )
    )

Archivo #1: Thread-124205675-Welcome to pol - Politically Incorrect.csv
Archivo #2: Thread-259848258-Not Subject.csv
Archivo #3: Thread-434008998-Humour Thread.csv
Archivo #4: Thread-434012818-Dear White People,.csv
Archivo #5: Thread-434014765-auspol aussie heros edition.csv
Archivo #6: Thread-434016208-Juden Peterstein.csv
Archivo #7: Thread-434021201-Not Subject.csv
Archivo #8: Thread-434024619-You will accept a black friend into your group and you will be happy..csv
Archivo #9: Thread-434026836-WE WON, YOU LOST!!.csv
Archivo #10: Thread-434027893-Not Subject.csv
Archivo #11: Thread-434028339-Cucks vs Nazis.csv
Archivo #12: Thread-434028622-Chuck Schumer just declared war against SCOTUS.csv
Archivo #13: Thread-434029177-If you eat out more than you cook at home, then you need to lower your tone when you talk to me.csv
Archivo #14: Thread-434029427-Britpol Queen Anne Edition.csv
Archivo #15: Thread-434030718-MEN ARE NOT DATING ANYMORE.csv
Archivo #16: Thread-434031086-Not Subject.csv

Los archivos contienen saltos de línea dentro de los campos, y las respuestas a un comentario raíz tienen el prefijo "(REPLY) >>", seguido del número del usuario. Esto se eliminará de las publicaciones.

In [41]:
def pre_procesar_dataset(dataset: pandas.DataFrame):

    # Se eliminan los saltos de línea y los nombres de usuario en las publicaciones/respuestas
    dataset['comment/reply'] = dataset['comment/reply'].str.replace(r'\n|\r', ' ', regex=True)
    dataset['comment/reply'] = dataset['comment/reply'].str.replace(
        '((\(REPLY\))?\s>>.{8})|>', '', regex=True)
    dataset['comment/reply'] = dataset['comment/reply'].str.lstrip()
    # Se retiran las columnas que no se usarán
    columnas = [
        'post_id',
        'subject',
        'name',
        'Name',
        'is_op?',
    ]
    dataset.drop(columns=columnas, inplace=True, errors='ignore')
    
    # Se renombran las columnas -> Esta adecuación se hace para darles un formato similar a los dataframes de Twitter,
    # para que en algún momento sean más sencillos de procesar bajo el mismo conjunto de instrucciones
    dataset.rename(
        columns={
            'date_time': 'Date',
            'comment/reply': 'Content',
            'url': 'URL'
        },
        inplace=True
    )

    # Se eliminan las filas que no tienen contenido
    dataset.dropna(subset=['Content'], inplace=True)

def obtener_hilo_desde_ruta(ruta: str):
    # Para conocer el hilo, se obtiene el nombre del directorio padre de la ruta
    nombre_hilo = os.path.basename(os.path.dirname(ruta))
    # Se eliminan los números y el guión del nombre del hilo
    nombre_hilo = re.sub(r'^([0-9]*\s-\s)|^(Thread-[0-9]*-)', '', nombre_hilo, )

    return nombre_hilo

In [42]:
# Se leen los archivos .csv encontrados
datasets = []

for ruta in rutas_archivos:

    dataset = pandas.read_csv(ruta)
    pre_procesar_dataset(dataset)

    datasets.append(
        {
            "hilo": obtener_hilo_desde_ruta(ruta),
            "datos": dataset,
        }
    )

In [35]:
# Observemos un ejemplo
datasets[4]['datos'].head()

,Date,Content,URL
0,"Jul-10-2023, 15:32:21",new usage figures show the number of calls mad...,http://boards.4chan.org/pol/thread/434014765/a...
1,"Jul-10-2023, 15:35:52",Based FEDPOL thread faggot kill yourself,http://boards.4chan.org/pol/thread/434014765/a...
2,"Jul-10-2023, 15:38:46",give it up mate cant you just tell me your bre...,http://boards.4chan.org/pol/thread/434014765/a...
3,"Jul-10-2023, 15:40:10",FAGGITS,http://boards.4chan.org/pol/thread/434014765/a...
4,"Jul-10-2023, 15:44:23",enough about you and your dad,http://boards.4chan.org/pol/thread/434014765/a...


In [43]:
# Agregamos una columna con el nombre del hilo
for dataset in datasets:
    dataset['datos']['Thread'] = dataset['hilo']

In [37]:
datasets[4]['datos'].head()

,Date,Content,URL,Thread
0,"Jul-10-2023, 15:32:21",new usage figures show the number of calls mad...,http://boards.4chan.org/pol/thread/434014765/a...,auspol aussie heros edition
1,"Jul-10-2023, 15:35:52",Based FEDPOL thread faggot kill yourself,http://boards.4chan.org/pol/thread/434014765/a...,auspol aussie heros edition
2,"Jul-10-2023, 15:38:46",give it up mate cant you just tell me your bre...,http://boards.4chan.org/pol/thread/434014765/a...,auspol aussie heros edition
3,"Jul-10-2023, 15:40:10",FAGGITS,http://boards.4chan.org/pol/thread/434014765/a...,auspol aussie heros edition
4,"Jul-10-2023, 15:44:23",enough about you and your dad,http://boards.4chan.org/pol/thread/434014765/a...,auspol aussie heros edition


Una vez preparados los datos, podemos guardar una copia de ellos para su futuro análisis.

In [38]:
# Creamos la ruta donde se guardarán los archivos
ruta_guardado = "./datos_limpios/4chan"
sufijo = "_cleaned.csv"

os.makedirs(ruta_guardado, exist_ok=True)
print(f'Ruta creada: {ruta_guardado}')

Ruta creada: ./datos_limpios/4chan


In [44]:
for dict_ in datasets:
    hilo = dict_['hilo']
    dataset = dict_['datos']
    nombre_guardado_full = f"{ruta_guardado}/{hilo}{sufijo}"
    dataset.to_csv(nombre_guardado_full, index=False)

Ahora que agregamos información del hilo al que pertenece, podemos juntar todas las publicaciones en un solo archivo.

In [45]:
dataset_full = pandas.concat([dict_['datos'] for dict_ in datasets])

ruta_dataset_full = f"{ruta_guardado}/dataset_4chan.csv"

# Comprobamos si el archivo ya existe. Si no existe, incluiremos la fila de encabezado
dataset_full_existe = True if os.path.isfile(ruta_dataset_full) else False
incluir_encabezado = False if dataset_full_existe else True

# Siempre se anexan los datos nuevos al archivo
dataset_full.to_csv(ruta_dataset_full, index=False, header=incluir_encabezado, mode='a')

Además, podemos leer una serie de palabras relevantes de un archivo, para después obtener únicamente publicaciones que las contienen.

In [97]:
ruta_archivo_palabras = './datos_p_filtrar/palabras_objetivo.txt'

with open(ruta_archivo_palabras, 'r') as archivo:
    palabras = archivo.readlines()
    palabras = [palabra.replace('\n', '') for palabra in palabras]
    print("Algunas de las palabras son: ", palabras[0:5])

Algunas de las palabras son:  ['suicide', 'suicidal', 'suic', 'self-harm', 'self-injury']


In [101]:
# Tomamos el dataset completo y recuperamos las publicaciones que contengan las palabras objetivo

def filtrar_dataset_sesion(dataset: pandas.DataFrame, palabras: list):

    # Quitamos las filas que no tienen contenido
    dataset.dropna(subset=['Content'], inplace=True)

    # Se crea una expresión regular que busca las palabras objetivo
    regex = '|'.join(palabras)

    print(regex)

    # Se filtra el dataset
    dataset_filtrado = dataset[
        dataset['Content'].str.contains(regex, case=False, regex=True)
    ]

    return dataset_filtrado

def filtrar_dataset_completo(ruta_dataset: str, palabras: list):

    # Se leen los datos
    dataset = pandas.read_csv(ruta_dataset)

    # Se filtra el dataset
    dataset_filtrado = filtrar_dataset_sesion(dataset, palabras)
    return dataset_filtrado

In [102]:
# Mantenemos las filas que contengan las palabras objetivo

# Descomentar esta línea si se trabajará con el dataset en memoria
# dataset_full_filtrado = filtrar_dataset_sesion(dataset_full, palabras)

# Descomentar estas líneas si se trabajará con el dataset completo
dataset_full_ruta = pandas.read_csv(f"{ruta_guardado}/dataset_4chan.csv")
dataset_full_filtrado = filtrar_dataset_completo(ruta_dataset_full, palabras)

suicide|suicidal|suic|self-harm|self-injury|self harm|self injury|hang myself|hung myself|kill myself|kills myself|killed myself|take my life|takes my life|want to die|wanted to die|wants to die|want death|wants death|wanted death|to be dead


In [105]:
dataset_full_filtrado.head()

,Date,Content,URL,Thread
383,"Jun-23-2023, 05:19:17",i am a BIG BLACK COCK BVLL now go commit suic...,http://boards.4chan.org/bant/thread/17918614/b...,_bant__AI_Waifus_General__121
387,"Jun-23-2023, 05:27:10",I won now go livestream your suicide,http://boards.4chan.org/bant/thread/17918614/b...,_bant__AI_Waifus_General__121
505,"Jun-23-2023, 08:01:39",Gladly not white but my brother is and he talk...,http://boards.4chan.org/bant/thread/17928132/m...,manlet_pajeets_stealing_Polish_women
934,"Jun-08-2023, 20:37:00",I went to Tokyo when I was 17 in the summer of...,http://boards.4chan.org/trv/thread/2478234/why...,No Subject
1802,"Jan-17-2023, 02:36:50","Ultimately, ending slavery is a moralfag white...",http://boards.4chan.org/e/thread/2669577/slave...,Slaves


In [107]:
# Escribimos el dataset filtrado, incluyendo el encabezado o no según corresponda

# Descomentar esta línea si se trabajará con el dataset en memoria
# modo = 'a'
# incluir_encabezado = False
# Descomentar estas líneas si se trabajará con el dataset completo
modo = 'w'
incluir_encabezado = True

dataset_full_filtrado.to_csv(f"{ruta_guardado}/dataset_4chan_keywords_only.csv", index=False, mode=modo, header=incluir_encabezado)

Para posteriores análisis, podemos decidir guardar los archivos procesados en la lista, para indicar que terminamos de correrlos por el pipeline.

In [17]:
# Anexamos los archivos procesados en esta sesión a la lista de archivos procesados

with open('./datos/lista_procesados.txt', 'a') as archivo:
    for ruta in rutas_archivos:
        hilo = obtener_hilo_desde_ruta(ruta)
        nombre_archivo = os.path.basename(ruta)
        archivo.write(f"{nombre_archivo}\n")